In [2]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.layers import Dropout


In [3]:
def load_data(image_dir, mask_dir, batch_size):
    image_filenames = [filename for filename in os.listdir(image_dir) if filename.endswith('.jpg')]
    num_images = len(image_filenames)
    
    for i in range(0, num_images, batch_size):
        print("Loading batch", (i // batch_size) + 1, "of", (num_images // batch_size) + 1)
        images_batch = []
        masks_batch = []
        batch_filenames = image_filenames[i:i+batch_size]
        
        for filename in batch_filenames:
            # Load image
            image_path = os.path.join(image_dir, filename)
            image = cv2.imread(image_path)
            image = cv2.resize(image, (2, 256))  # Resize image if necessary
            images_batch.append(image)
            
            # Load corresponding mask
            mask_filename = filename[:-4] + '_segmentation.png'
            mask_path = os.path.join(mask_dir, mask_filename)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (256, 256))  # Resize mask if necessary
            masks_batch.append(mask)
        
        yield (np.array(images_batch)/255,(np.array(masks_batch)/255).astype(int))

In [4]:
def load_image(image_dir, mask_dir, batch_size):
    data_generator = load_data(image_dir, mask_dir, batch_size)
    images_all = []
    masks_all = []
    num_images = len([filename for filename in os.listdir(image_dir) if filename.endswith('.jpg')])
    num_batches = (num_images + batch_size - 1) // batch_size
    
    for _ in range(num_batches):
        data = next(data_generator)
        images, masks = data
        images_all.append(images)
        masks_all.append(masks)
    
    images_all = np.concatenate(images_all, axis=0)
    masks_all = np.concatenate(masks_all, axis=0)
    print("Total images shape:", images_all.shape)
    print("Total masks shape:", masks_all.shape)
    return images_all, masks_all

In [5]:
train_images, train_masks = load_image('/kaggle/input/isic2018/ISIC2018_Task1-2_Training_Input/ISIC2018_Task1-2_Training_Input','/kaggle/input/isic2018/ISIC2018_Task1_Training_GroundTruth/ISIC2018_Task1_Training_GroundTruth',128)

Loading batch 1 of 21
Loading batch 2 of 21
Loading batch 3 of 21
Loading batch 4 of 21
Loading batch 5 of 21
Loading batch 6 of 21
Loading batch 7 of 21
Loading batch 8 of 21
Loading batch 9 of 21
Loading batch 10 of 21
Loading batch 11 of 21
Loading batch 12 of 21
Loading batch 13 of 21
Loading batch 14 of 21
Loading batch 15 of 21
Loading batch 16 of 21
Loading batch 17 of 21
Loading batch 18 of 21
Loading batch 19 of 21
Loading batch 20 of 21
Loading batch 21 of 21
Total images shape: (2594, 256, 256, 3)
Total masks shape: (2594, 256, 256)


In [6]:
test_images, test_masks = load_image('/kaggle/input/isic2018/ISIC2018_Task1-2_Test_Input/ISIC2018_Task1-2_Test_Input','/kaggle/input/isic2018/ISIC2018_Task1_Test_GroundTruth/ISIC2018_Task1_Test_GroundTruth',64)

Loading batch 1 of 16
Loading batch 2 of 16
Loading batch 3 of 16
Loading batch 4 of 16
Loading batch 5 of 16
Loading batch 6 of 16
Loading batch 7 of 16
Loading batch 8 of 16
Loading batch 9 of 16
Loading batch 10 of 16
Loading batch 11 of 16
Loading batch 12 of 16
Loading batch 13 of 16
Loading batch 14 of 16
Loading batch 15 of 16
Loading batch 16 of 16
Total images shape: (1000, 256, 256, 3)
Total masks shape: (1000, 256, 256)


In [7]:
test_masks[0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
val_images, val_masks = load_image('/kaggle/input/isic2018/ISIC2018_Task1-2_Validation_Input/ISIC2018_Task1-2_Validation_Input','/kaggle/input/isic2018/ISIC2018_Task1_Validation_GroundTruth/ISIC2018_Task1_Validation_GroundTruth',64)

Loading batch 1 of 2
Loading batch 2 of 2
Total images shape: (100, 256, 256, 3)
Total masks shape: (100, 256, 256)


In [24]:
def unet_model(input_shape, learning_rate=1e-4, dropout_rate=0.2, weight_decay=1e-5):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(dropout_rate)(pool1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(dropout_rate)(pool2)
    
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(dropout_rate)(pool3)
    
    # Bottleneck
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(conv4)
    conv4 = Dropout(dropout_rate)(conv4)
    
    # Decoder
    up5 = Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv4)
    up5 = concatenate([up5, conv3], axis=3)
    conv5 = Conv2D(256, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(up5)
    conv5 = Conv2D(256, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(conv5)
    conv5 = Dropout(dropout_rate)(conv5)
    
    up6 = Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv5)
    up6 = concatenate([up6, conv2], axis=3)
    conv6 = Conv2D(128, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(up6)
    conv6 = Conv2D(128, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(conv6)
    conv6 = Dropout(dropout_rate)(conv6)
    
    up7 = Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv6)
    up7 = concatenate([up7, conv1], axis=3)
    conv7 = Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(up7)
    conv7 = Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(conv7)
    
    outputs = Conv2D(1, 1, activation='sigmoid')(conv7)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    # Compile model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy')
    
    return model

In [12]:
from sklearn.metrics import jaccard_score, confusion_matrix

def threshold_jaccard_index(predicted_masks, ground_truth_masks, threshold=0.65):
    scores = []
    for i in range(len(predicted_masks)):
        # Binarize the predicted mask using a threshold
        #binary_predicted_mask = (predicted_masks[i] > 0.5).astype(int) * 255
        
        # Compute the Jaccard index between the binary predicted mask and ground truth mask
        jaccard_index = jaccard_score(ground_truth_masks[i].flatten(), predicted_masks[i].flatten(),labels=[0, 1], average="binary")
        overall_jaccard_index = np.mean(jaccard_index)  # Compute the overall Jaccard index for the entire mask
        if overall_jaccard_index < threshold:
            scores.append(0)
        else:
            scores.append(overall_jaccard_index)
    return np.mean(scores)

In [13]:
def jaccard_index(predicted_mask, ground_truth_mask):
    return jaccard_score(ground_truth_mask.flatten(), predicted_mask.flatten())

In [28]:
learning_rate = 0.0000005
dropout_rate = 0.2
weight_decay = 1e-5


model = unet_model(input_shape=(256, 256, 3), learning_rate=learning_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)

# Callbacks
checkpoint = ModelCheckpoint("unet_model.keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=7, verbose=1, mode='min', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, min_lr=0.0005, verbose=1, mode='min')

# Train the model
history = model.fit(train_images, train_masks, batch_size=16, epochs=100, validation_split = 0.2, callbacks=[checkpoint, early_stopping, reduce_lr])

Epoch 1/100
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step - loss: 0.7121
Epoch 1: val_loss improved from inf to 0.69623, saving model to unet_model.keras
130/130 ━━━━━━━━━━━━━━━━━━━━ 74s 428ms/step - loss: 0.7120 - val_loss: 0.6962 - learning_rate: 5.0000e-07
Epoch 2/100
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step - loss: 0.6904
Epoch 2: val_loss improved from 0.69623 to 0.67639, saving model to unet_model.keras
130/130 ━━━━━━━━━━━━━━━━━━━━ 39s 303ms/step - loss: 0.6904 - val_loss: 0.6764 - learning_rate: 5.0000e-07
Epoch 3/100
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step - loss: 0.6696
Epoch 3: val_loss improved from 0.67639 to 0.65617, saving model to unet_model.keras
130/130 ━━━━━━━━━━━━━━━━━━━━ 39s 303ms/step - loss: 0.6695 - val_loss: 0.6562 - learning_rate: 5.0000e-07
Epoch 4/100
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step - loss: 0.6473
Epoch 4: val_loss improved from 0.65617 to 0.63137, saving model to unet_model.keras
130/130 ━━━━━━━━━━━━━━━━━━━━ 39s 303ms/step - loss: 0.6472 - val_l

In [29]:
predicted_masks = model.predict(test_images)
predicted_masks = (predicted_masks > 0.5).astype(int)

32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 178ms/step


In [32]:
threshold_jaccard = threshold_jaccard_index(predicted_masks, test_masks)
print("Threshold Jaccard Index:", threshold_jaccard)

Threshold Jaccard Index: 0.3750459036925851


In [9]:
def custom_unet_model(input_shape, learning_rate=1e-4, dropout_rate=0.2, weight_decay=1e-5, num_filters=64, num_layers=4):
    inputs = Input(input_shape)
    
    # Encoder
    conv_layers = []
    pool_layers = []
    x = inputs
    for _ in range(num_layers):
        conv = Conv2D(num_filters, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(x)
        conv = Conv2D(num_filters, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(conv)
        pool = MaxPooling2D(pool_size=(2, 2))(conv)
        pool = Dropout(dropout_rate)(pool)
        conv_layers.append(conv)
        pool_layers.append(pool)
        x = pool
    
    # Bottleneck
    conv = Conv2D(num_filters * 2, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(x)
    conv = Conv2D(num_filters * 2, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(conv)
    conv = Dropout(dropout_rate)(conv)
    
    # Decoder
    for conv_layer, pool_layer in zip(reversed(conv_layers), reversed(pool_layers)):
        up = Conv2DTranspose(num_filters, 2, strides=(2, 2), padding='same')(conv)
        up = concatenate([up, conv_layer], axis=3)
        conv = Conv2D(num_filters, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(up)
        conv = Conv2D(num_filters, 3, activation='relu', padding='same', kernel_regularizer=l2(weight_decay))(conv)
        conv = Dropout(dropout_rate)(conv)
    
    outputs = Conv2D(1, 1, activation='sigmoid')(conv)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    # Compile model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy')
    
    return model

model = custom_unet_model(input_shape=(256, 256, 3))


In [10]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

def build_model(hp):
    model = custom_unet_model(
        input_shape=(256, 256, 3),
        learning_rate=hp.Float('learning_rate', 1e-6, 1e-2, sampling='log'),
        dropout_rate=hp.Float('dropout_rate', 0.1, 0.5, step=0.1),
        weight_decay=hp.Float('weight_decay', 1e-6, 1e-3, sampling='log'),
        num_filters=hp.Choice('num_filters', [32, 64, 128]),
        num_layers=hp.Int('num_layers', 3, 5)
    )
    return model

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='hyperparameter_tuning',
    project_name='custom_unet'
)

tuner.search(train_images, train_masks, validation_split = 0.2, epochs=10, batch_size=16)
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
print(best_hyperparameters.values)


Trial 10 Complete [00h 02m 47s]
val_loss: 0.37028780579566956

Best val_loss So Far: 0.37028780579566956
Total elapsed time: 01h 07m 37s
Best Hyperparameters:
{'learning_rate': 6.901714677917224e-05, 'dropout_rate': 0.2, 'weight_decay': 7.792822719960166e-05, 'num_filters': 32, 'num_layers': 4}


/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 94 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [13]:
predicted_masks = best_model.predict(test_images)
predicted_masks = (predicted_masks > 0.5).astype(int)
threshold_jaccard = threshold_jaccard_index(predicted_masks, test_masks)
print("Threshold Jaccard Index:", threshold_jaccard)

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step
Threshold Jaccard Index: 0.4924629954628306


In [42]:
model = custom_unet_model(
        input_shape=(256, 256, 3),
        learning_rate=7e-05,
        dropout_rate=0.2,
        weight_decay=7.792822719960166e-05,
        num_filters=32,
        num_layers=5,
    )

In [43]:
checkpoint = ModelCheckpoint("unet_model.keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.0005, verbose=1, mode='min')

# Train the model
history = model.fit(train_images, train_masks, batch_size=16, epochs=100, validation_split = 0.2, callbacks=[checkpoint, early_stopping, reduce_lr])

Epoch 1/100
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step - loss: 0.5663
Epoch 1: val_loss improved from inf to 0.47907, saving model to unet_model.keras
130/130 ━━━━━━━━━━━━━━━━━━━━ 64s 279ms/step - loss: 0.5659 - val_loss: 0.4791 - learning_rate: 7.0000e-05
Epoch 2/100
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.4217
Epoch 2: val_loss improved from 0.47907 to 0.46393, saving model to unet_model.keras
130/130 ━━━━━━━━━━━━━━━━━━━━ 12s 89ms/step - loss: 0.4216 - val_loss: 0.4639 - learning_rate: 7.0000e-05
Epoch 3/100
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.3651
Epoch 3: val_loss did not improve from 0.46393
130/130 ━━━━━━━━━━━━━━━━━━━━ 11s 88ms/step - loss: 0.3650 - val_loss: 0.4856 - learning_rate: 7.0000e-05
Epoch 4/100
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 1.3016
Epoch 4: val_loss did not improve from 0.46393
130/130 ━━━━━━━━━━━━━━━━━━━━ 11s 87ms/step - loss: 1.3096 - val_loss: 3.6266 - learning_rate: 7.0000e-05
Epoch 5/100
130/130 ━━━━━━━━━━━━━━━━━━━━ 0

In [24]:
predicted_masks = model.predict(test_images)
predicted_masks = (predicted_masks > 0.5).astype(int)
threshold_jaccard = threshold_jaccard_index(predicted_masks, test_masks)
print("Threshold Jaccard Index:", threshold_jaccard)

32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step
Threshold Jaccard Index: 0.5090880172650178
